# SQL: Шаблоны и функции для аналитиков

**PostgreSQL** — стандарт для аналитики.

## Содержание
1. [Порядок SELECT-запроса](#select-order)  
2. [CTE (Common Table Expressions)](#cte)  
3. [Оконные функции](#window-functions)  
4. [JOIN и подзапросы](#join-subqueries)  
5. [Агрегация и фильтрация](#aggregation)  
6. [Оптимизация запросов](#optimization)  

---

## <a name="select-order"></a>1. Порядок SELECT-запроса

```sql
SELECT      -- 1. Что выбираем
FROM        -- 2. Откуда
JOIN        -- 3. Соединения
WHERE       -- 4. Фильтр строк
GROUP BY    -- 5. Группировка
HAVING      -- 6. Фильтр групп
ORDER BY    -- 7. Сортировка
LIMIT       -- 8. Ограничение
```
> **Правило:** Пишем в **логическом порядке**, движок выполняет в **порядке оптимизатора**.

> **Зачем:** Читаемость + контроль порядка выполнения движка.

## <a name="cte"></a>2. CTE (Common Table Expressions)

```sql
WITH monthly_sales AS (
    SELECT 
        region,
        DATE_TRUNC('month', sale_date) AS month,
        SUM(amount) AS total
    FROM sales
    WHERE sale_date >= '2025-01-01'
    GROUP BY 1, 2
)
SELECT * FROM monthly_sales ORDER BY total DESC;
```

> **Зачем:** Читаемость, многошаговые расчёты, переиспользование.

## <a name="window-functions"></a>3. Оконные функции

```sql
-- % от общего по региону
SELECT 
    region,
    SUM(amount) AS region_total,
    ROUND(
        100.0 * SUM(amount) / SUM(SUM(amount)) OVER (), 
        2
    ) AS pct_total
FROM expenses
GROUP BY region;

-- Топ-3 с RANK
SELECT * FROM (
    SELECT 
        store,
        SUM(sales) AS total,
        RANK() OVER (ORDER BY SUM(sales) DESC) AS rnk
    FROM sales 
    GROUP BY store
) WHERE rnk <= 3;
```

> **Зачем:** Ранжирование, % от общего, сравнение с соседями — без `GROUP BY`.

## <a name="join-subqueries"></a>4. JOIN и подзапросы

```sql
-- LEFT JOIN
SELECT c.name, o.amount
FROM customers c
LEFT JOIN orders o ON c.id = o.customer_id;

-- Подзапрос в HAVING
SELECT region, SUM(amount)
FROM expenses
GROUP BY region
HAVING SUM(amount) > (
    SELECT AVG(total) FROM (
        SELECT SUM(amount) AS total FROM expenses GROUP BY region
    ) sub
);
```

> **Зачем:** `JOIN` — соединение таблиц. Подзапросы — фильтр по агрегации (редко, только если JOIN не подходит).

## <a name="aggregation"></a>5. Агрегация и фильтрация

```sql
-- CASE + COALESCE
SELECT 
    region,
    COALESCE(category, 'unknown') AS cat, → если category NULL → 'unknown'
    CASE 
        WHEN amount > 10000 THEN 'high'
        ELSE 'low'
    END AS tier
FROM expenses;
```

> **Зачем:** Условные столбцы, обработка NULL, категоризация.\n
> - `COALESCE` — безопасно от NULL",
> - `CASE` — категоризация (как IF в Excel)"

## <a name="optimization"></a>6. Оптимизация запросов

```sql
-- Правильно: фильтр ДО агрегации
SELECT region, SUM(amount)
FROM expenses
WHERE amount > 1000 AND date >= '2025-01-01'
GROUP BY region;

-- Проверить план
EXPLAIN ANALYZE SELECT ...;
```

> **Зачем:** Скорость на больших данных. Избегай перегрузки DWH.

### Справка: Что хорошо / плохо

| Хорошо | Плохо |
|--------|-------|
| `WHERE` → `GROUP BY` → `HAVING` | `SELECT *` в продакшене |
| `JOIN` вместо подзапросов | Подзапросы в `SELECT` (пересчёт на строку) |
| Индексы на `date`, `category_id` | `OR` вместо `IN` / `UNION` |
| `EXPLAIN ANALYZE` перед пушем | `LIMIT` без `ORDER BY` |


> **Подзапросы:** можно, если без вариантов. `JOIN` + `CTE` — всегда быстрее.


**EXPLAIN ANALYZE перед пушем**  
Запускаешь `EXPLAIN ANALYZE SELECT ...` локально перед коммитом в продакшен.  
Показывает реальный план выполнения (время, сколько строк читает).  

> **Зачем:** увидишь, не тормозит ли запрос на миллионах строк.

**LIMIT без ORDER BY**  
`SELECT * FROM sales LIMIT 10` → случайные 10 строк (каждый раз разные).  
Плохо: результат непредсказуемый — нельзя доверять в отчётах.  
Правильно: `ORDER BY date DESC LIMIT 10` — последние 10.

**Индексы на `date`, `category_id`**  
`CREATE INDEX ON expenses(date);`  

> **Зачем:** фильтр `WHERE date = '2025-11-06'` → за 1 мс, а не за 10 сек.

**OR вместо IN / UNION**  
`WHERE region = 'Москва' OR region = 'Питер'` → медленнее.  

> **Лучше:** `WHERE region IN ('Москва', 'Питер')` — оптимизатор использует индекс.

---
**Источники:** PostgreSQL docs, LeetCode SQL.  
*Добавляйте свои шаблоны через PR.*